In [ ]:
from fastai.vision.all import *
import pathlib

In [ ]:
# Download images from Bing
# Review images after download and remove any inappropriate items

#from bing_image_downloader import downloader
#downloader.download("./images/pileated woodpecker", limit=150,  output_dir='images', adult_filter_off=True, force_replace=False, timeout=60, verbose=True)
#downloader.download("./images/red bellied woodpecker", limit=150,  output_dir='images', adult_filter_off=True, force_replace=False, timeout=60, verbose=True)

In [ ]:
# load the images
path = Path('images')
fns = get_image_files(path/'pileated woodpecker')
fns # fns is a fastai L object which combines functionality from both lists and numpy arrays

In [ ]:
# create a datablock which fastai will be used to create the dataloaders object for training
woodpeckers = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # A tuple specifying the type of independent data and dependent data 
    get_items=get_image_files, # A function that can load the data - get_image_files is a fastai function that loads all images in a given path
    splitter=RandomSplitter(valid_pct=.2, seed=42), # A class that splits the data into a training and validation set
    get_y=parent_label, # A function that gets the dependent variable - parent_label labels each image with its parent folder name
    item_tfms=Resize(128), # Transforms to apply to the data - since downloaded images are different sizes, resize them
)

In [ ]:
# create the DataLoaders object
dls = woodpeckers.dataloaders(path)

In [ ]:
# show a few samples from the training set
dls.train.show_batch(max_n=4, nrows=1)

In [ ]:
# show a few samples from the validation set
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# apply final transforms
woodpeckers = woodpeckers.new(item_tfms=RandomResizedCrop(224, min_scale=.5), batch_tfms=aug_transforms())
dls = woodpeckers.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True) # unique=True shows the same image repeated with different ResizedRandomCrop transforms applied

In [ ]:
# train the model
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(8)

In [ ]:
# analyze the predictions
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
# to further analyze the mistakes, sort the images by loss
interp.plot_top_losses(8, nrows=8)

In [ ]:
# export the model
learn.export()

In [ ]:
# load the model
learn_inf = load_learner('export.pkl')

In [ ]:
# make a prediction
#learn_inf.predict('images/pileated woodpecker/Image_1.jpeg')
learn_inf.predict('images/red bellied woodpecker/Image_101.jpeg')

# output is: class label, class id, probabilities